In [70]:
import pandas as pd
import numpy as np

In [71]:
AT16 = ['6DO1','6OS0','6OS1','6OS2']
AT14 = ['4YAY','4ZUD']
AT1 = AT14 + AT16
arr = ['6TKO','6U1N','6UP7','6PWC']

In [72]:
hyd_aci = pd.read_csv('/home/kestr/AT1/data/hydrogen_contacts.csv')
repulsion = pd.read_csv('/home/kestr/AT1/data/charge_repulsion.csv')
other_contacts = pd.read_csv('/home/kestr/AT1/data/contacts_ion_and_hydrophobic.csv')
total = pd.read_csv('/home/kestr/AT1/data/model_res_dict.csv')
ms = pd.read_csv('/home/kestr/AT1/data/mutants_suggestions.csv')

aligned_cutted_arr_path = '../pdb/2_aligned/cutted_arr'
AT1_arr_path = '../pdb/2_aligned/AT1'
arr_path = '../pdb/1_BW/arr'

In [73]:
hyd_aci=hyd_aci[['gpcr','arr','chain','Cluster_ID','res']]
hyd_aci['type'] = 'HY'
repulsion=repulsion[['gpcr','arr','chain','Cluster_ID','res']]
repulsion['type'] = 'RE'
contacts = pd.concat([other_contacts, hyd_aci, repulsion])
contacts_R = contacts[contacts.chain == 'R']
contacts_A = contacts[contacts.chain == 'A']
contacts_A = contacts_A.merge(total,left_on=['arr','chain','res'],right_on=['pdb_code','chain_id','residue_number'])
contacts_R = contacts_R.merge(total,left_on=['gpcr','chain','res'],right_on=['pdb_code','chain_id','residue_number'])

In [114]:
# Отрисовка кластеров контактов  до рекомендаций Саши
script = f'''
    reinitialize
    preset.publication(selection='all')
    '''

for AT1_code in AT1:
    for arr_code in arr:
        script += f'''
    load ../{aligned_cutted_arr_path}/{AT1_code}_{arr_code}.pdb, {AT1_code}_{arr_code}
    alter chain T, chain='R'
    util.cbag(selection='chain R')
    util.cbak(selection='chain A')
    hide all
    '''
        con_R=contacts_R[contacts_R.gpcr==AT1_code][contacts_R.arr==arr_code]
        con_A=contacts_A[contacts_A.gpcr==AT1_code][contacts_A.arr==arr_code]
        for BW in con_R.b_factor.unique():
            cont_R_BW = con_R[con_R.b_factor==BW]
            for clu in cont_R_BW['Cluster_ID'].unique():
                cont_R_BW_clu = cont_R_BW[cont_R_BW.Cluster_ID==clu]
                for typec in cont_R_BW_clu['type'].unique():
                    arr_reses = con_A[con_A.Cluster_ID==clu][con_A.type==typec]
                    cont_R_BW_clu_type = cont_R_BW_clu[cont_R_BW_clu.type==typec]
                    flag = 0
                    for arr_res in arr_reses.res.unique():
                        gpcr_res = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_number.iloc[0])
                        gpcr_res_name = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_name.iloc[0])
                        if(flag == 0):
                            script+=f'''select init_res, {AT1_code}_{arr_code} and chain R and resi {gpcr_res}\n'''
                            flag = 1
                        script+=f'''select init_res, {AT1_code}_{arr_code} and chain A and resi {arr_res} or init_res\n'''
                    script+=f'''
                        show sticks, init_res
                        h_add (ele n+o+s+se) and init_res
                        contacts {AT1_code}_{arr_code} and chain A and init_res, {AT1_code}_{arr_code} and chain R and init_res, result="contacts_{AT1_code}_{arr_code}", cutoff=3.1, bigcutoff=3.6
                        label n. N & ! chain R and init_res, resi+resn
                        label n. N & chain R and init_res, resi+resn+' %1.2f'%b
                        set label_position,(0,1.5,4)
                        show cartoon, {AT1_code}_{arr_code}
                        set cartoon_transparency, 0.5
                        orient init_res
                        viewport 1024,768
                        bg_color white
                        set label_size, 25
                        set label_font_id, 7
                        move z,-3
                        ray
                        png ../../images/res_clusters/{typec}/{BW}_{AT1_code}_{arr_code}_{clu}.png
                        hide all


                        '''

with open(f'../scripts/3_images/images_clu.pml', 'w') as f:
            f.write(script)

/tmp/ipykernel_4029973/1639547522.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  con_R=contacts_R[contacts_R.gpcr==AT1_code][contacts_R.arr==arr_code]
/tmp/ipykernel_4029973/1639547522.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  con_A=contacts_A[contacts_A.gpcr==AT1_code][contacts_A.arr==arr_code]
/tmp/ipykernel_4029973/1639547522.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  arr_reses = con_A[con_A.Cluster_ID==clu][con_A.type==typec]


In [1]:
# Отрисовка кластеров контактов после рекомендаций Саши
script = f'''
    reinitialize
    preset.publication(selection='all')
    '''

for AT1_code in AT1:
    for arr_code in arr:
        script += f'''
    load ../{aligned_cutted_arr_path}/{AT1_code}_{arr_code}.pdb, {AT1_code}_{arr_code}
    alter chain T, chain='R'
    util.cbag(selection='chain R')
    util.cbak(selection='chain A')
    util.cnc
    '''
        con_R=contacts_R[contacts_R.gpcr==AT1_code][contacts_R.arr==arr_code]
        con_A=contacts_A[contacts_A.gpcr==AT1_code][contacts_A.arr==arr_code]
        for BW in con_R.b_factor.unique():
            cont_R_BW = con_R[con_R.b_factor==BW]
            for clu in cont_R_BW['Cluster_ID'].unique():
                cont_R_BW_clu = cont_R_BW[cont_R_BW.Cluster_ID==clu]
                for typec in cont_R_BW_clu['type'].unique():
                    arr_reses = con_A[con_A.Cluster_ID==clu][con_A.type==typec]
                    cont_R_BW_clu_type = cont_R_BW_clu[cont_R_BW_clu.type==typec]
                    flag = 0
                    for arr_res in arr_reses.res.unique():
                        gpcr_res = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_number.iloc[0])
                        gpcr_res_name = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_name.iloc[0])
                        if(flag == 0):
                            script+=f'''select init_res, {AT1_code}_{arr_code} and chain R and resi {gpcr_res}\n'''
                            flag = 1
                        script+=f'''select init_res, {AT1_code}_{arr_code} and chain A and resi {arr_res} or init_res\n'''
                    script+=f'''
                        show sticks, init_res
                        hide everything, (name C+N+O) and not resn PRO
                        hide everything, (name C+O) and resn PRO
                        contacts {AT1_code}_{arr_code} and chain A and init_res, {AT1_code}_{arr_code} and chain R and init_res, result="contacts_{AT1_code}_{arr_code}", cutoff=3.1, bigcutoff=3.6
                        label n. N & ! chain R and init_res, resi+resn
                        label n. N & chain R and init_res, resi+resn+' %1.2f'%b
                        set label_position,(0,1.5,4)
                        show cartoon, {AT1_code}_{arr_code}
                        set cartoon_transparency, 0.5
                        orient init_res
                        viewport 1024,768
                        bg_color white
                        set label_size, 25
                        set label_font_id, 7
                        move z,-5
                        ray
                        png ../../images/res_clusters/{typec}/{BW}_{AT1_code}_{arr_code}_{clu}.png
                        hide all

                        '''

with open(f'../scripts/3_images/images_clu.pml', 'w') as f:
            f.write(script)
            # png ../../images/res_clusters/mutations_suggestions/.png

SyntaxError: f-string: empty expression not allowed (2823932505.py, line 60)

In [75]:
# отрисовка возможных мутаций
for iter in range(ms.shape[0]):
    script = f'''
    reinitialize
    preset.publication(selection='all')
    '''
    arr_code = ms.gpcr_orig_pdb[iter]
    res_arr = ms.arr_res_number[iter]
    gpcr_arr_res = total[total.b_factor == ms.gpcr_orig_res_number[iter]][total.pdb_code == arr_code]['residue_number'].iloc[0]
    script += f'''
    load ../{arr_path}/{arr_code}.pdb, {arr_code}
    util.cbay chain A & {arr_code}
    util.cbaw chain R & {arr_code}
    # set cartoon_side_chain_helper, on
    show cartoon, {arr_code}
    select init_res, {arr_code} and chain A and (resi {res_arr})
    select init_res, {arr_code} and chain R and resi {gpcr_arr_res} or init_res
    '''
    for AT1_code in AT1:
        if(AT1_code in AT14):
            color = 'k'
        else:
            color = 'c'
        flag=1
        try:
            AT1_res = total[total.b_factor == ms.gpcr_orig_res_number[iter]][total.pdb_code == AT1_code]['residue_number'].iloc[0]
        except:
            flag = 0
        if(flag==1):
            script += f'''
        load ../{aligned_cutted_arr_path}/{AT1_code}_{arr_code}.pdb, {AT1_code}_{arr_code}
        alter chain T, chain='R'
        remove {AT1_code}_{arr_code} and chain A
        util.cba{color} chain R & {AT1_code}_{arr_code}
        select init_res, {AT1_code}_{arr_code} and chain R and resi {AT1_res}  or init_res
        '''
    i='contact'
    script+=f'''
                        show sticks, init_res
                        label n. CB & ! chain R and init_res, resi+resn
                        label first (n. N & ! chain R and init_res), '{arr_code}'
                        label (n. CA & chain R and init_res and {arr_code}) , resi+resn+' %1.2f'%b
                        set label_color, grey20, (n. CA & chain R and init_res and {arr_code})
                        label first (n. CA & chain R and init_res and ! {arr_code}) , resi+resn+' %1.2f'%b
                        set label_color, violetpurple, first (n. CA & chain R and init_res and ! {arr_code})
                        contacts chain R and init_res, chain A and init_res, result="contacts_{AT1_code}_{arr_code}", cutoff=3.1, bigcutoff=3.6


    # select {i}, resi {res_arr} and {arr_code} and ! chain R
    # select {i}_round, {i} around 3.6 and {arr_code} and chain R
    # select {i}_orig, {i}_round around 3.6 and {arr_code} and ! chain R
    # select {i}_orig_res, byres {i}_round around 3.6
    # label ({i}_round or {i}_orig) and ! (name C+CA+CB+CD+CZ+CG), name
    # show lines, {i}_orig_res
    # label (n. CA & chain R and {i}_orig_res and {arr_code}) , resi+resn+' %1.2f'%b
    # set label_color, grey20, (n. CA & chain R and {i}_orig_res and {arr_code})
    # label first (n. CA & chain R and {i}_orig_res and ! {arr_code}) , resi+resn+' %1.2f'%b
    # set label_color, violetpurple, first (n. CA & chain R and {i}_orig_res and ! {arr_code})

                        set label_position,(0,1.5,5)
                        orient init_res
                        set cartoon_transparency, 0.5
                        viewport 1024,768
                        bg_color white
                        set label_size, 16
                        set label_font_id, 7
                        move z,-6
    '''
    with open(f'../scripts/4_mutations_2/{ms.gpcr_orig_res_number[iter]}_{ms.gpcr_orig_res_name[iter]}_{ms.AT1_res_name[iter]}.pml', 'w') as f:
        f.write(script)
        # png /home/kestr/AT1/images/mutations_suggestions/5.68_GLU_ALA.png

/tmp/ipykernel_2504299/626731503.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gpcr_arr_res = total[total.b_factor == ms.gpcr_orig_res_number[iter]][total.pdb_code == arr_code]['residue_number'].iloc[0]
/tmp/ipykernel_2504299/626731503.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  AT1_res = total[total.b_factor == ms.gpcr_orig_res_number[iter]][total.pdb_code == AT1_code]['residue_number'].iloc[0]
